# Sources

Upstox - https://upstox.com/developer/api-documentation/instruments/

NSE200 - https://www.nseindia.com/products-services/indices-nifty200-index

Vlookup - "C:\Users\perry\OneDrive\nifty200\ind_nifty200list.xlsx"

In [1]:
# open xlsx with nifty200 symbols and upstox instrument key
import pandas as pd

# Specify the file path
file_path = 'ind_nifty200list.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Print the DataFrame
# print(df)


In [2]:
from datetime import date, timedelta, datetime
import requests

def datesort(crow):
    dt = crow[0]
    return int(datetime.fromisoformat(dt).timestamp())

def get_6m_returns(instkey):
    ed = str(date.today())
    sd = str(date.today() - timedelta(weeks=26))
    url = f"https://api-v2.upstox.com/historical-candle/{instkey}/month/{ed}/{sd}"
    h = {
        "Api-Version": "2.0",
        "Accept": "application/json",
        "Authorization": "***REMOVED***",
    }

    # https://upstox.com/developer/api-documentation/get-historical-candle-data/
    resp = requests.get(url, headers=h)
    if resp.status_code != 200:
        print(f"inst: {instkey} err: {resp}")
        return None
    rjson = resp.json()
    
    if rjson["status"] != "success":
        raise Exception(rjson["status"])
    
    # print(f"{rjson}")
    candles = rjson["data"]["candles"]
    candles.sort(key=datesort, reverse=True)
    # for c in candles:
    #     print(f"date: {c[0]} close: {c[4]}")

    if candles:
        start_price = candles[-1][4]
        end_price = candles[0][4]
        return (end_price - start_price)/start_price
    return 0

# print(get_12m_returns("NSE_EQ|INE084A01016"))
# print(get_12m_returns("NSE_EQ|INE498L01015"))

In [4]:
print(get_6m_returns("NSE_EQ|INE121A08PJ0"))

0


In [ ]:
# Sort C200 by previous 6 months returns decreasing

import pandas as pd

sym_returns = dict()

def apply_12m_returns(row):
    # print(row)
    sym_returns[row['Symbol']] = get_6m_returns(row['instrument_key'])

df = pd.read_excel('ind_nifty200list.xlsx')
_ = df.apply(apply_12m_returns, axis='columns')

t200 = []
for k,v in sorted(sym_returns.items(), key=lambda x: x[1], reverse=True):
    t200.append({"symbol": k, "gain": v})

print(t200)


[{'symbol': 'PAYTM', 'gain': 1.1121344119477286}, {'symbol': 'BSE', 'gain': 0.8175446748368858}, {'symbol': 'COFORGE', 'gain': 0.4570601597420678}, {'symbol': 'KALYANKJIL', 'gain': 0.3850144667265291}, {'symbol': 'PERSISTENT', 'gain': 0.3365004892194886}, {'symbol': 'DIXON', 'gain': 0.30487916934668763}, {'symbol': 'LUPIN', 'gain': 0.29805409072686345}, {'symbol': 'DIVISLAB', 'gain': 0.29444462574378066}, {'symbol': 'HCLTECH', 'gain': 0.25890654973965477}, {'symbol': 'MFSL', 'gain': 0.2545538746526705}, {'symbol': 'MANKIND', 'gain': 0.24906068006763113}, {'symbol': 'ZOMATO', 'gain': 0.24017750299162338}, {'symbol': 'BHARTIHEXA', 'gain': 0.23503603885929178}, {'symbol': 'BAJAJHLDNG', 'gain': 0.2286776980524234}, {'symbol': 'PAGEIND', 'gain': 0.2277160617759631}, {'symbol': 'POLICYBZR', 'gain': 0.2014313830738952}, {'symbol': 'PIIND', 'gain': 0.20103981572885812}, {'symbol': 'APOLLOHOSP', 'gain': 0.19976720500509237}, {'symbol': 'VOLTAS', 'gain': 0.1995652764570031}, {'symbol': 'HDFCLIFE

In [6]:
for s in t200:
    print(f"{s}")

{'symbol': 'PAYTM', 'gain': 1.1121344119477286}
{'symbol': 'BSE', 'gain': 0.8175446748368858}
{'symbol': 'COFORGE', 'gain': 0.4570601597420678}
{'symbol': 'KALYANKJIL', 'gain': 0.3850144667265291}
{'symbol': 'PERSISTENT', 'gain': 0.3365004892194886}
{'symbol': 'DIXON', 'gain': 0.30487916934668763}
{'symbol': 'LUPIN', 'gain': 0.29805409072686345}
{'symbol': 'DIVISLAB', 'gain': 0.29444462574378066}
{'symbol': 'HCLTECH', 'gain': 0.25890654973965477}
{'symbol': 'MFSL', 'gain': 0.2545538746526705}
{'symbol': 'MANKIND', 'gain': 0.24906068006763113}
{'symbol': 'ZOMATO', 'gain': 0.24017750299162338}
{'symbol': 'BHARTIHEXA', 'gain': 0.23503603885929178}
{'symbol': 'BAJAJHLDNG', 'gain': 0.2286776980524234}
{'symbol': 'PAGEIND', 'gain': 0.2277160617759631}
{'symbol': 'POLICYBZR', 'gain': 0.2014313830738952}
{'symbol': 'PIIND', 'gain': 0.20103981572885812}
{'symbol': 'APOLLOHOSP', 'gain': 0.19976720500509237}
{'symbol': 'VOLTAS', 'gain': 0.1995652764570031}
{'symbol': 'HDFCLIFE', 'gain': 0.1906562

In [7]:
# select top 40 (T40) and top 20 (T20) from it.

t40 = t200[:40]
t20 = t200[:20]

In [9]:
# Open portfolio.csv, which contains all symbols held currently, call it HXX

import pandas as pd

df = pd.read_csv("portfolio6.csv")

sell = []
buy = []
hold = []

for i, r in df.iterrows():
    if not any(t['symbol'] == r['Symbol'] for t in t40):
        sell.append(r['Symbol'])
    elif any(t['symbol'] == r['Symbol'] for t in t20):
        buy.append(r['Symbol'])
    else:
        hold.append(r['Symbol'])

for t in t20:
    if t['symbol'] not in buy:
        buy.append(t['symbol'])

print(f"buy: {buy}")
print(f"sell: {sell}")
print(f"hold: {hold}")
    

buy: ['PAYTM', 'BSE', 'COFORGE', 'KALYANKJIL', 'PERSISTENT', 'DIXON', 'LUPIN', 'DIVISLAB', 'HCLTECH', 'MFSL', 'MANKIND', 'ZOMATO', 'BHARTIHEXA', 'BAJAJHLDNG', 'PAGEIND', 'POLICYBZR', 'PIIND', 'APOLLOHOSP', 'VOLTAS', 'HDFCLIFE']
sell: []
hold: []


In [ ]:
# Update portfolio.csv

import pandas as pd

df = pd.read_csv("portfolio6.csv")

for s in sell:
    df = df[df['Symbol'] != s]

for b in buy:
    if b not in df['Symbol'].values:
        r = pd.DataFrame({'Symbol': [b], 'Units': [0]})
        df = pd.concat([df, r], ignore_index=True)

# Save the updated portfolio once at the end
df.to_csv('portfolio6.csv', index=False)
print(f"Portfolio updated with {len(buy)} buys and {len(sell)} sells")